In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
current_year = 2023

In [3]:
market = Market()
spec = ADatabase("btc_spec")
umod = UniversalModeler()

In [4]:
factors = [str(x) for x in range(14)]

In [5]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
training_sets = []

In [8]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
ticker_data = prices
ticker_data.sort_values("date",ascending=True,inplace=True)
ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
for i in range(14):
    ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
ticker_data.dropna(inplace=True)
ticker_data["ticker"] = "BTC"
ticker_data["y"] = ticker_data["adjclose"].shift(-1)
ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
ticker_data.dropna(inplace=True)
ticker_data = ticker_data[included_columns]
training_sets.append(ticker_data)

In [9]:
data = pd.concat(training_sets)

In [10]:
data = data.dropna()

In [11]:
spec.connect()
for modeler in [umod]:
    for year in tqdm(range(current_year,current_year+1)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - 4)].reset_index(drop=True)
        set_name = "models"
        stuff = modeler.recommend_model(training_slice,factors)
spec.disconnect()

  0%|                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

Epoch 1/10
5/5 [==============================] - 1s 2ms/step - loss: 1945464704.0000 - mean_absolute_percentage_error: 153.6333
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 620132608.0000 - mean_absolute_percentage_error: 79.8121
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 105843136.0000 - mean_absolute_percentage_error: 27.1960
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 116317896.0000 - mean_absolute_percentage_error: 32.2930
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 165036864.0000 - mean_absolute_percentage_error: 38.9405
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 105290752.0000 - mean_absolute_percentage_error: 30.5379
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 51776708.0000 - mean_absolute_percentage_error: 20.2546
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 44643848.0000 - mean_absolute_percentage

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.11s/it]


In [12]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_absolute_percentage_error
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-03 11:19:38         2522
metadata.json                                  2023-03-03 11:19:38           64
variables.h5                                   2023-03-03 11:19:39        84160


In [13]:
spec.connect()
models = spec.store("models",stuff)
spec.disconnect()